## Deliverable 2. Create a Customer Travel Destinations Map.

In [22]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [23]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Saint-Louis,SN,16.3333,-15.0000,70.99,20,26,11.70,scattered clouds
1,1,Albany,US,42.6001,-73.9662,9.00,50,3,3.00,clear sky
2,2,Mayo,US,38.8876,-76.5119,30.99,51,90,11.50,overcast clouds
3,3,Tiksi,RU,71.6872,128.8694,-30.53,85,100,5.50,overcast clouds
4,4,Kaitangata,NZ,-46.2817,169.8464,57.99,72,24,8.01,few clouds


In [24]:
# 2. Prompt the user to enter minimum and maximum temperature criteria
min_temp=float(input("What is the minimum temperature you would like for your vacation?"))
max_temp=float(input("What is the minimum temperature you would like for your vacation?"))

What is the minimum temperature you would like for your vacation?65
What is the minimum temperature you would like for your vacation?85


In [26]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Saint-Louis,SN,16.3333,-15.0000,70.99,20,26,11.70,scattered clouds
5,5,Rikitea,PF,-23.1203,-134.9692,80.47,70,31,9.06,scattered clouds
6,6,Quelimane,MZ,-17.8786,36.8883,75.92,91,99,1.25,overcast clouds
12,12,Carauari,BR,-4.8828,-66.8958,74.14,95,100,2.33,overcast clouds
13,13,Hilo,US,19.7297,-155.0900,75.20,73,90,6.91,moderate rain


In [27]:
preferred_cities_df.tail()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
676,676,Sechura,PE,-5.5569,-80.8222,76.98,64,81,13.65,broken clouds
678,678,Wanaka,NZ,-44.7000,169.1500,67.51,42,1,2.91,clear sky
679,679,Pochutla,MX,15.7432,-96.4661,78.80,83,75,6.91,broken clouds
680,680,Callaguip,PH,18.0714,120.4867,82.40,44,75,10.36,broken clouds
681,681,Dingle,PH,10.9995,122.6711,76.93,85,100,17.02,overcast clouds


In [28]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                4
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [29]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
new_preferred_cities_df=preferred_cities_df.dropna()
new_preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Saint-Louis,SN,16.3333,-15.0000,70.99,20,26,11.70,scattered clouds
5,5,Rikitea,PF,-23.1203,-134.9692,80.47,70,31,9.06,scattered clouds
6,6,Quelimane,MZ,-17.8786,36.8883,75.92,91,99,1.25,overcast clouds
12,12,Carauari,BR,-4.8828,-66.8958,74.14,95,100,2.33,overcast clouds
13,13,Hilo,US,19.7297,-155.0900,75.20,73,90,6.91,moderate rain


In [30]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = new_preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Saint-Louis,SN,70.99,scattered clouds,16.3333,-15.0000,
5,Rikitea,PF,80.47,scattered clouds,-23.1203,-134.9692,
6,Quelimane,MZ,75.92,overcast clouds,-17.8786,36.8883,
12,Carauari,BR,74.14,overcast clouds,-4.8828,-66.8958,
13,Hilo,US,75.20,moderate rain,19.7297,-155.0900,
14,Mataura,NZ,70.48,clear sky,-46.1927,168.8643,
17,Bima,ID,81.48,heavy intensity rain,-8.4667,118.7167,
21,Cape Town,ZA,70.00,broken clouds,-33.9258,18.4232,
23,Butaritari,KI,80.46,light rain,3.0707,172.7902,
26,Kavaratti,IN,80.24,clear sky,10.5669,72.6420,


In [31]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.") 

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [32]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Saint-Louis,SN,70.99,scattered clouds,16.3333,-15.0000,
5,Rikitea,PF,80.47,scattered clouds,-23.1203,-134.9692,Pension Maro'i
6,Quelimane,MZ,75.92,overcast clouds,-17.8786,36.8883,Hotel Flamingo
12,Carauari,BR,74.14,overcast clouds,-4.8828,-66.8958,Hotel Fellip
13,Hilo,US,75.20,moderate rain,19.7297,-155.0900,Hilo Hawaiian Hotel
14,Mataura,NZ,70.48,clear sky,-46.1927,168.8643,Ellie's Villa
17,Bima,ID,81.48,heavy intensity rain,-8.4667,118.7167,Lambitu Hotel
21,Cape Town,ZA,70.00,broken clouds,-33.9258,18.4232,"Radisson Blu Hotel Waterfront, Cape Town"
23,Butaritari,KI,80.46,light rain,3.0707,172.7902,Isles Sunset Lodge
26,Kavaratti,IN,80.24,clear sky,10.5669,72.6420,Hotels in Lakshadweep Islands


In [33]:
hotel_df.tail(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
662,Fonte Boa,BR,76.03,broken clouds,-2.4959,-66.2662,
665,Wewak,PG,84.40,light rain,-3.5534,143.6268,In Wewak Boutique Hotel
667,Phalaborwa,ZA,71.60,broken clouds,-23.9430,31.1411,Bothabelo B&B
670,Vung Tau,VN,76.77,overcast clouds,10.3460,107.0843,Seaside Resort
672,Jacqueville,CI,77.00,light rain,5.2052,-4.4146,Hotel Le Martin Pecheur
676,Sechura,PE,76.98,broken clouds,-5.5569,-80.8222,Hospedaje Costa Bella
678,Wanaka,NZ,67.51,clear sky,-44.7000,169.1500,Wanaka Homestead Lodge and Cottages
679,Pochutla,MX,78.80,broken clouds,15.7432,-96.4661,Hotel Posada San Jose
680,Callaguip,PH,82.40,broken clouds,18.0714,120.4867,Pangil Beach Resort
681,Dingle,PH,76.93,overcast clouds,10.9995,122.6711,SEGAYA PROPERTY


In [34]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"]!=""]
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Rikitea,PF,80.47,scattered clouds,-23.1203,-134.9692,Pension Maro'i
6,Quelimane,MZ,75.92,overcast clouds,-17.8786,36.8883,Hotel Flamingo
12,Carauari,BR,74.14,overcast clouds,-4.8828,-66.8958,Hotel Fellip
13,Hilo,US,75.20,moderate rain,19.7297,-155.0900,Hilo Hawaiian Hotel
14,Mataura,NZ,70.48,clear sky,-46.1927,168.8643,Ellie's Villa
...,...,...,...,...,...,...,...
676,Sechura,PE,76.98,broken clouds,-5.5569,-80.8222,Hospedaje Costa Bella
678,Wanaka,NZ,67.51,clear sky,-44.7000,169.1500,Wanaka Homestead Lodge and Cottages
679,Pochutla,MX,78.80,broken clouds,15.7432,-96.4661,Hotel Posada San Jose
680,Callaguip,PH,82.40,broken clouds,18.0714,120.4867,Pangil Beach Resort


In [35]:
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Rikitea,PF,80.47,scattered clouds,-23.1203,-134.9692,Pension Maro'i
6,Quelimane,MZ,75.92,overcast clouds,-17.8786,36.8883,Hotel Flamingo
12,Carauari,BR,74.14,overcast clouds,-4.8828,-66.8958,Hotel Fellip
13,Hilo,US,75.20,moderate rain,19.7297,-155.0900,Hilo Hawaiian Hotel
14,Mataura,NZ,70.48,clear sky,-46.1927,168.8643,Ellie's Villa


In [36]:
# 8a. Create the output File (CSV)
output_data_file = "clean_hotel_df.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [37]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [38]:
# 11a. Add a marker layer for each city to the map. 
#City = clean_hotel_df["City"]
max_temp = clean_hotel_df("Max Temp")
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=2.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=info_box_template)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

TypeError: 'DataFrame' object is not callable